<a href="https://colab.research.google.com/github/andylolu2/simpleGEMM/blob/master/colab/simpleGEMM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [12]:
!git clone --recurse-submodules https://github.com/andylolu2/simpleGEMM.git
%cd simpleGEMM

Cloning into 'simpleGEMM'...
remote: Enumerating objects: 34, done.
remote: Counting objects: 100% (34/34), done.
remote: Compressing objects: 100% (25/25), done.
remote: Total 34 (delta 18), reused 19 (delta 8), pack-reused 0
Receiving objects: 100% (34/34), 14.39 KiB | 14.39 MiB/s, done.
Resolving deltas: 100% (18/18), done.
Submodule 'src/cutlass' (https://github.com/NVIDIA/cutlass.git) registered for path 'cutlass'
Cloning into '/content/simpleGEMM/cutlass'...
remote: Enumerating objects: 26417, done.        
remote: Counting objects: 100% (7566/7566), done.        
remote: Compressing objects: 100% (976/976), done.        
remote: Total 26417 (delta 6938), reused 6603 (delta 6590), pack-reused 18851        
Receiving objects: 100% (26417/26417), 41.14 MiB | 25.30 MiB/s, done.
Resolving deltas: 100% (20145/20145), done.
Submodule path 'cutlass': checked out '19f3cc33f1642b490ed7126ea0141f79c0045527'
/content/simpleGEMM


In [24]:
!mkdir build
!nvcc \
    --include-path ./ \
    --include-path cutlass/include \
    --generate-code=arch=compute_75,code=[compute_75,sm_75] \
    --expt-relaxed-constexpr \
    -forward-unknown-to-host-compiler \
    -std=c++17 \
    -O3 \
    -o build/main \
    main.cu

mkdir: cannot create directory ‘build’: File exists


In [25]:
!./build/main 4096 4096 4096 1000

Time elapse: 4524.4ms
TFLOPS: 30.3773


In [23]:
# Compare against PyTorch (CuBLAS)
import torch

M, N, K = 4096, 4096, 4096
iters = 1000

A = torch.randn(M, K, dtype=torch.float16, device="cuda")
B = torch.randn(N, K, dtype=torch.float16, device="cuda").T

start = torch.cuda.Event(enable_timing=True)
end = torch.cuda.Event(enable_timing=True)
start.record()
for _ in range(iters):
    C = A @ B
end.record()
torch.cuda.synchronize()

total_duration = start.elapsed_time(end)  # in ms
tflops = 2 * M * N * K * iters / (total_duration / 1000) / 1e12
print(f"Time elapse: {total_duration:.2f}ms")
print(f"TFLOPS: {tflops:.4f}")

Time elapse: 7318.69ms
TFLOPS: 18.7792


> TODO: Why is PyTorch so slow?